# Exploration Silver

L'exploration des données va consiter à observer la qualité des données et à observer différentes statistiques à propos de celle-ci.

### 1. Import librairies

L'import des librairies nécessaire à l'xploration des données.

In [9]:
from pathlib import Path

import duckdb

from sirene_pipeline.config import settings

# 1. On trouve la racine du projet (le dossier parent de 'notebooks')
# __get_working_dir() n'existe pas en notebook, on utilise Path.cwd()
current_dir = Path.cwd()
root_path = current_dir.parent if current_dir.name == "notebooks" else current_dir

# 2. On configure DuckDB
con = duckdb.connect()

# 3. On boucle sur les fichiers Silver (etablissements et unites_legales)
silver_dir = root_path / settings.silver.output_dir

print(f"📂 Looking for Silver files in: {silver_dir}\n")

for dataset_name in ["etablissements", "unites_legales"]:
    silver_path = silver_dir / f"{dataset_name}_silver.parquet"

    if silver_path.exists():
        # Utilisation de as_posix() pour DuckDB sur Windows
        sql_path = silver_path.as_posix()

        # Création de la vue SQL pour l'exploration
        con.execute(
            f"CREATE OR REPLACE VIEW {dataset_name} AS SELECT * FROM read_parquet('{sql_path}')"
        )

        print(f"✅ View '{dataset_name}' created from {silver_path.name}")

        # Affichage des colonnes pour confirmer
        cols = con.execute(f"DESCRIBE {dataset_name}").df()
        print(f"   📊 {len(cols)} columns detected.")
    else:
        print(f"⚠️ File not found: {silver_path}")

📂 Looking for Silver files in: c:\Users\atexier\Documents\Projet_Sirene\data\silver

✅ View 'etablissements' created from etablissements_silver.parquet
   📊 14 columns detected.
✅ View 'unites_legales' created from unites_legales_silver.parquet
   📊 13 columns detected.


### 2. Analyse de données

Ce script permet de regarder l'unicité des données et d'explorer les différentes distribution de certains champs.

In [ ]:
# On définit les colonnes clés à monitorer pour chaque dataset
monitor_config = {
    "etablissements": {
        "id": "siret",
        "cat": ["departement", "secteur_activite", "etatAdministratifEtablissement"],
        "num": ["age_entreprise"],
    },
    "unites_legales": {
        "id": "siren",
        "cat": ["secteur_activite", "economieSocialeSolidaireUniteLegale"],
        "num": ["age_entreprise"],
    },
}

for table, cols in monitor_config.items():
    print(f"\n{'=' * 20} QUALITY REPORT: {table.upper()} {'=' * 20}")

    # 1. DATA QUALITY : Nulls & Unicity
    quality_query = f"""
        SELECT 
            COUNT(DISTINCT {cols["id"]}) as unique_ids,
            COUNT(*) - COUNT({cols["id"]}) as missing_ids,
            ROUND(
                AVG(CASE WHEN secteur_activite = 'In' THEN 1 ELSE 0 END) * 100, 
                2
            ) as pct_indeterminate_sector
        FROM {table}
    """
    df_quality = con.execute(quality_query).df()

    # Check for duplicates
    is_unique = df_quality["total_rows"][0] == df_quality["unique_ids"][0]
    status_icon = "✅" if is_unique else "🚨"

    print("\n🛡️ Integrity Check:")
    print(f"- Total Rows: {df_quality['total_rows'][0]:,}")
    print(f"- Unique IDs: {df_quality['unique_ids'][0]:,} {status_icon}")
    print(f"- Missing IDs: {df_quality['missing_ids'][0]}")

    # 2. CATEGORICAL STATS : Distribution
    print("\n📊 Categorical Distributions:")
    for cat_col in cols["cat"]:
        dist = con.execute(f"""
            SELECT {cat_col}, COUNT(*) as count, 
                   ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM {table}), 1) as pct
            FROM {table}
            GROUP BY 1 ORDER BY count DESC LIMIT 5
        """).df()
        print(f"\nTop 5 for '{cat_col}':")
        display(dist)

    # 3. NUMERICAL STATS : Descriptive
    print("\n📈 Numerical Profiles:")
    num_stats = con.execute(f"""
        SELECT 
            MIN({cols["num"][0]}) as min_age,
            ROUND(AVG({cols["num"][0]}), 1) as avg_age,
            MAX({cols["num"][0]}) as max_age,
            APPROX_QUANTILE({cols["num"][0]}, 0.5) as median_age
        FROM {table}
        WHERE {cols["num"][0]} >= 0
    """).df()
    display(num_stats)


==================== QUALITY REPORT: ETABLISSEMENTS ====================

🛡️ Integrity Check:
- Total Rows: 186
- Unique IDs: 186 ✅
- Missing IDs: 0

📊 Categorical Distributions:

Top 5 for 'department':


,department,count,pct
0,91,73,39.2
1,75,44,23.7
2,92,24,12.9
3,93,13,7.0
4,77,12,6.5



Top 5 for 'activity_sector':


,activity_sector,count,pct
0,85,49,26.3
1,70,27,14.5
2,88,19,10.2
3,84,13,7.0
4,74,11,5.9



Top 5 for 'etatAdministratifEtablissement':


...

,activity_sector,count,pct
0,84,3063,38.3
1,55,429,5.4
2,64,361,4.5
3,67,283,3.5
4,68,244,3.1



Top 5 for 'economieSocialeSolidaireUniteLegale':


,economieSocialeSolidaireUniteLegale,count,pct
0,N,7874,98.4
1,O,126,1.6



📈 Numerical Profiles:


,min_age,avg_age,max_age,median_age
0,14,54.8,126,48
